# DNA Sequence and Reverse Complement Database Preparation

This notebook focuses on the preparation of a comprehensive database, consisting of DNA sequences and their corresponding reverse complements. While not essential for running the evaluation itself, the code presented here is instrumental in curating the dataset leveraged during the evaluation process.

## Source Genome Data

We have employed the v3.0 genome data of Populus trichocarpa (black cottonwood) for this purpose. This specific version of the genome is known for its detailed representation, making it an ideal choice for our analysis.

## Citation for Genome Data
Tuskan, Gerald A., et al. "The genome of black cottonwood, Populus trichocarpa (Torr. & Gray)." Science 313.5793 (2006): 1596-1604.

## Data Availability
The file Ptrichocarpa_444_v3.0.fa can be accessed and downloaded from the Pytozome-next.jgi.doe.gov portal.

## Code

Feel free to explore the code cells below, where we randomly extract specific lengths of sequences from the defined chromosomes and generate their reverse complements. The final result is a pandas dataframe, neatly organizing the extracted sequences, their origin, positions, and corresponding reverse complements.

### Produce dataframe of sequences and reverse complements for ground truth

In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd
import random

random.seed(123)

# Define X and Y
X = 100
Y = range(10, 511)

# Read the FASTA file
fasta_file = "Ptrichocarpa_444_v3.0.fa"

# Store chromosome sequences and info, considering only chromosomes 1-19
chromosomes_info = []
for record in SeqIO.parse(fasta_file, "fasta"):
    if record.id.startswith('Chr') and int(record.id[3:]) <= 19:
        chromosomes_info.append({'id': record.id, 'length': len(record.seq), 'sequence': str(record.seq)})

results = []

while len(results) < X:
    # Randomly select a chromosome and desired sequence length
    chrom_info = random.choice(chromosomes_info)
    y = random.choice(Y)

    if y > chrom_info['length']:
        continue

    sequence = chrom_info['sequence']

    # Randomly select a start position and check for 'N'
    start = random.randint(0, chrom_info['length'] - y - 1)
    end = start + y

    while 'N' in sequence[start:end]:
        start = random.randint(0, chrom_info['length'] - y - 1)
        end = start + y

    extracted_sequence = sequence[start:end]
    reverse_complement = str(Seq(extracted_sequence).reverse_complement())

    # Append to results
    results.append({
        'chromosome': chrom_info['id'],
        'start': start,
        'end': end,
        'length': y,
        'sequence': extracted_sequence,
        'revcomp': reverse_complement
    })

# Convert to a dataframe
df = pd.DataFrame(results)

# Print the dataframe
print(df)


    chromosome     start       end  length  \
0        Chr02   2925499   2925646     147   
1        Chr14   3613820   3613966     146   
2        Chr02  17991954  17992158     204   
3        Chr18  11435290  11435470     180   
4        Chr02   4531776   4531867      91   
..         ...       ...       ...     ...   
495      Chr09   7304962   7305098     136   
496      Chr05  19537658  19538138     480   
497      Chr17   6350618   6350823     205   
498      Chr07     91920     92401     481   
499      Chr03  21249029  21249417     388   

                                              sequence  \
0    ATGTCCACCATAAAACCAGAGCTACATAGCCATGGAAAGTTCTAAC...   
1    CACTTCGCCACCATGTCCAATCCAATTTGGAATATTATCTTTAAGA...   
2    AGATTCAATCCATTCAGGTCATTTGCTACTCCTTATTCTTGTTGGC...   
3    TCTACTGTAATTGTTTGTAGTGACGTTTGCTGTCGGCCTCCTTTTG...   
4    AAGATAAATATATTTTACATAATATTAATGAGCGATATATATTTAT...   
..                                                 ...   
495  TTTTTCAATGATTTGAAAAAAAAGAGGGTTTGACTC

### Prepare `yaml` and `jsonl` files for eval

In [2]:
import os
import json

# Create the reverse_complement directory if it doesn't exist
os.makedirs('reverse_complement', exist_ok=True)

# Define a function to convert a single row to the JSONL format
def row_to_jsonl_format(row):
    system_content = ("You are a bioinformatics assistant. Provide the reverse complement"
                      " to each DNA sequence given, with no additional text or formatting.")
    content = {
        "input": [
            {"role": "system", "content": system_content},
            {"role": "user", "content": row['sequence']}
        ],
        "ideal": row['revcomp']
    }
    return json.dumps(content)

# Apply the function to the DataFrame and write to file
with open('reverse_complement/reverse_complement.jsonl', 'w') as file:
    for _, row in df.iterrows():
        file.write(row_to_jsonl_format(row) + '\n')


In [14]:
import yaml

# YAML content for reverse complement
yaml_content = {
    "reverse-complement": {
        "id": "reverse-complement.s1.simple-v0",
        "description": "Test the model's ability to provide the reverse complement to DNA sequences.",
        "disclaimer": "The performance of models tends to depend on sequence length and repetition, with more artifacts appearing for longer sequences and polynucleotides.",
        "metrics": ["accuracy"]
    },
    "reverse-complement.s1.simple-v0": {
        "class": "evals.elsuite.basic.match:Match",
        "args": {
            "samples_jsonl": "reverse_complement/samples.jsonl"
        }
    }
}

# Write the YAML content to a file
with open('reverse_complement.yaml', 'w') as file:
    yaml.dump(yaml_content, file, default_flow_style=False)


## Place `json` and `yaml` in appropriate places in the `evals` repository

In [15]:
import os
data_path = "../evals/evals/registry/data/reverse_complement/"
isExist = os.path.exists(data_path)
if not isExist:
   os.makedirs(data_path)

In [16]:
with open('../evals/evals/registry/evals/reverse_complement.yaml', 'w') as file:
    yaml.dump(yaml_content, file, default_flow_style=False)
    
with open(data_path + 'samples.jsonl', 'w') as file:
    for _, row in df.iterrows():
        file.write(row_to_jsonl_format(row) + '\n')